
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 2.1 DEMO: Implementing Delta Sharing (Databricks-to-Databricks) \[Provider]

## Overview
In this demo, we will implement Delta Sharing to securely share data between Databricks workspaces. This demonstration is split into two parts:

**Provider Notebook (This Notebook):** We will create a Unity Catalog with sample customer and sales data, configure Delta Sharing by creating a share, set up a recipient organization, and generate an activation link for secure access.

**Recipient Notebook:** The receiving organization (using Databricks) will mount the share in a local catalog and query the shared tables directly without copying data, and perform analytics using the shared datasets.

This demo showcases the Databricks-to-Databricks (D2D) sharing pattern, where both the provider and recipient use Databricks workspaces. You'll see how Delta Sharing enables secure, live data sharing without data duplication or complex ETL processes.

### Learning Objectives
By the end of this demo, you will be able to:
1. Create and configure Unity Catalog resources (catalogs, schemas, and tables) suitable for Delta Sharing.
2. Implement Delta Sharing by creating a share and adding tables to it for external access.
3. Create and configure recipients to control who can access your shared data.
4. Understand the provider's role in the Delta Sharing architecture and how to manage shared data access.
5. Verify and monitor shared data configurations using SQL commands in the provider workspace.

## Background

**Scenario:**
You are a data provider company (*"Acme Corp"*) that wants to share customer analytics data with partner organizations using Delta Sharing. In this notebook, you'll set up the provider side of Delta Sharing by creating the data, configuring shares, and generating access credentials for your recipients.

In [0]:
%run ./Includes/Demo-Setup-2

## Inspect catalog, schema and tables

The setup script has set up a catalog, schema and tables for our fictitious company which will act as the __provider__, lets examine these now:

In [0]:
-- get current context
SELECT current_catalog(), current_schema()

In [0]:
-- show tables in our schema
SHOW TABLES;

In [0]:
-- show the data in the customers table
SELECT * FROM customers;

In [0]:
-- show the data in the sales_transactions table
SELECT * FROM sales_transactions;

## Step 1: Create a Delta Share

Now we'll create a share and add our tables to it. A share is a logical grouping of tables that can be shared with recipients.

In [0]:
-- Create a new share
CREATE SHARE IF NOT EXISTS internal_retail
COMMENT 'Share containing customer and sales data for partner analysis';

-- View all shares
SHOW SHARES;

## Step 2: Add Assets to the Share

Add the `customers` and `sales_transactions` tables to the share. Once added, these tables become available for recipients to query through the Delta Sharing protocol.

In [0]:
-- Add tables to the share
ALTER SHARE internal_retail 
ADD TABLE acme_corp.retail.customers
COMMENT 'Customer master data';

ALTER SHARE internal_retail 
ADD TABLE acme_corp.retail.sales_transactions
COMMENT 'Sales transaction history';

-- View what's in the share
SHOW ALL IN SHARE internal_retail

In [0]:
/* to remove a table from a share
ALTER SHARE internal_retail
REMOVE TABLE acme_corp.retail.sales_transactions;
*/

## Step 3: Create a Recipient (Databricks-to-Databricks)

Create a recipient for Databricks-to-Databricks sharing. For D2D sharing, the recipient needs to provide their metastore ID.  This can be found by selecting 
⚙️ the __Delta Sharing__ from __Catalog Explorer__:
<br />
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/recipient%20-%20get%20metastore%20id.png?raw=true"
    alt="Get Metastore ID"
  >
</div>
<br />
<br />
The recipient then copies their Metastore ID, using the __Copy Sharing Identifier__ option shown here, they will need to supply this to you.
<br />
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/recipient%20-%20get%20metastore%20id%202.png?raw=true"
    alt="Get Metastore ID"
  >
</div>

In [0]:
-- The recipient can also get there sharing identifier (metastore id) using system.information_schema
SELECT global_metastore_id FROM system.information_schema.metastores;

In [0]:
-- or they can get the identifier using a describe statement (Global Metastore ID)
DESCRIBE METASTORE;

In [0]:
-- Create a Databricks-to-Databricks recipient
CREATE RECIPIENT IF NOT EXISTS west_division
USING ID 'aws:us-west-2:adc48801-0d35-4e94-9cbc-0ff7f29ce611'
COMMENT 'Recipient for D2D Delta Sharing demo';

-- View all recipients
SHOW RECIPIENTS;

In [0]:
-- Describe the recipient to get the activation link
DESCRIBE RECIPIENT west_division;

## Step 4: Grant the Recipient Access to the Share

Grant the recipient `SELECT` permissions on the share to enable data access. This connects the recipient to the share, allowing them to query the shared tables through Delta Sharing.

In [0]:
-- Grant the share to the recipient
GRANT SELECT ON SHARE internal_retail TO RECIPIENT west_division;

-- Verify grants on the share
SHOW GRANTS ON SHARE internal_retail;

In [0]:
-- Check which shares a specific recipient has access to
SHOW GRANTS TO RECIPIENT west_division;

## Step 5: Mount the Share in a Catalog (Recipient)
Switch over to the recipient workspace and follow the instructions in the __2.1 DEMO Implementing Delta Sharing (D2D)__ recipient notebook.  

⚠️ The recipient who will mount the share needs to have the `USE PROVIDER` privilege for your provider OR be a __Metastore Admin__, the provider name can be found by selecting 
⚙️ the __Delta Sharing__ from __Catalog Explorer__ on the __Provider workspace__, see below:
<br />
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/provider%20-%20get%20provider%20name.png?raw=true"
    alt="Get Provider Name"
  >
</div>
<br />
<br />
The name can also be changed at the provider using <b>Edit Organization Name</b>.

## Step 6: Monitor and Manage the Share

As Delta Sharing uses __credential vending__ to issue short lived credentials to authorized calling users, we can see the inbound activity for assets in a share using the __`system.access.audit `__ table as shown below:

In [0]:
-- query share usage from the provider side
SELECT 
event_time,
request_params.table_full_name,
request_params.recipient_name,
request_params.share_name
FROM system.access.audit 
WHERE action_name = 'generateTemporaryTableCredential'
AND user_agent = 'Delta Sharing'
ORDER BY event_time DESC;

The result would look like...  

| event_time | table_full_name | recipient_name | share_name |
|---|---|---|---|
| 2025-10-20T05:25:34.814+00:00 | acme_corp.retail.sales_transactions | west_division | internal_retail |
| 2025-10-20T04:49:59.124+00:00 | acme_corp.retail.customers | west_division | internal_retail |
| .. | .. | .. | .. |

The detailed query history will also be available from the recipients workspace using the __`system.query.history`__ table.  If the shared asset is a (non materialized) view, you would be able to see history in the __`system.query.history`__ table on the providers side as well, including compute usage.  

More information about auditing Delta Sharing can be found at [__Audit and monitor data sharing__](https://docs.databricks.com/aws/en/delta-sharing/audit-logs).

## Summary

✅ **What we accomplished:**

1. Created a Delta Share and added tables to it
2. Created a Databricks-to-Databricks recipient
3. Granted the share to the recipient

**Key Benefits:**
- 🚀 **No Data Copying**: Query data directly from the provider
- 🔒 **Secure**: Provider controls access and can revoke at any time
- ⚡ **Real-time**: Always access the latest data from the provider
- 💰 **Cost-effective**: No storage costs for recipients
- 🔧 **Standard SQL**: Use familiar SQL syntax for queries

&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>